## Pytorch Tutorial 따라하기 (Quick Start Version)

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader ## Dataset을 순회 가능한 객체로 감싼다. 
from torchvision import datasets
from torchvision.transforms import ToTensor

In [6]:
## 공개 데이터셋에서 Train 데이터를 받습니다.
training_data = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor()
)

## 공개 데이터셋에서 Test 데이터를 받습니다.
test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor()
)

In [7]:
batch_size = 64

# Data Loader를 생성합니다.
train_dataloder = DataLoader(training_data,batch_size = batch_size)
test_dataloader = DataLoader(test_data,batch_size = batch_size)

for X,y in test_dataloader:
    print(f"[N C H W] shape의 X : {X.shape}")
    print(f"shape of y :{y.shape}")
    break

[N C H W] shape의 X : torch.Size([64, 1, 28, 28])
shape of y :torch.Size([64])


## Model 만들기

In [24]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# model을 정의합니다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)        

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Model 매개변수 최적화하기

In [25]:
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [26]:
def train(dataloader,model,loss_fn, optimizer):
    size=len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)

        # predict error calculate
        pred = model(X)
        loss = loss_fn(pred, y)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, currect = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{currect:>5d} / {size:>5d}]")  

In [32]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)

    num_batchs = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    
    with torch.no_grad():
        for X,y in dataloader:
            X,y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        
        test_loss /= num_batchs
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
        return correct, test_loss

In [35]:
epoch = 5
for t in range(epoch):

    print(f"Epoch {t +1}\n =====================================")
    train(train_dataloder,model, loss_fn,optimizer)
    acc, loss = test(test_dataloader, model,loss_fn)
print(f"Train Result: Epoch{t} Accuracy:{acc} Avg loss: {loss}")
print("Done!")

Epoch 1
loss: 0.626866 [    0 / 60000]
loss: 0.731875 [ 6400 / 60000]
loss: 0.496069 [12800 / 60000]
loss: 0.740531 [19200 / 60000]
loss: 0.652936 [25600 / 60000]
loss: 0.633558 [32000 / 60000]
loss: 0.711945 [38400 / 60000]
loss: 0.717071 [44800 / 60000]
loss: 0.703355 [51200 / 60000]
loss: 0.662936 [57600 / 60000]
Test Error: 
 Accuracy: 76.9%, Avg loss: 0.662731 

Epoch 2
loss: 0.607315 [    0 / 60000]
loss: 0.713495 [ 6400 / 60000]
loss: 0.480387 [12800 / 60000]
loss: 0.727765 [19200 / 60000]
loss: 0.641872 [25600 / 60000]
loss: 0.623037 [32000 / 60000]
loss: 0.695366 [38400 / 60000]
loss: 0.707620 [44800 / 60000]
loss: 0.693163 [51200 / 60000]
loss: 0.650237 [57600 / 60000]
Test Error: 
 Accuracy: 77.4%, Avg loss: 0.649612 

Epoch 3
loss: 0.589541 [    0 / 60000]
loss: 0.696552 [ 6400 / 60000]
loss: 0.466161 [12800 / 60000]
loss: 0.715963 [19200 / 60000]
loss: 0.631752 [25600 / 60000]
loss: 0.613698 [32000 / 60000]
loss: 0.679892 [38400 / 60000]
loss: 0.699281 [44800 / 60000]
loss